<a href="https://colab.research.google.com/github/JayThibs/Weak-Supervised-Learning-Case-Study/blob/main/text_classifier/notebooks/03_dbpedia_14_snorkel_dataset_labeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install snorkel --quiet
!pip install datasets --quiet
!pip install spacy --quiet

     |████████████████████████████████| 153kB 10.3MB/s 
     |████████████████████████████████| 3.8MB 8.7MB/s 
     |████████████████████████████████| 1.8MB 63.4MB/s 
ERROR: tensorflow 2.4.1 has requirement tensorboard~=2.4, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 225kB 12.6MB/s 
     |████████████████████████████████| 245kB 26.7MB/s 
     |████████████████████████████████| 112kB 32.6MB/s 


In [2]:
import pandas as pd
from snorkel.labeling import labeling_function
from datasets import load_dataset
import re

In [3]:
dbpedia_dataset = load_dataset('dbpedia_14')

Dataset d_bpedia14 downloaded and prepared to /root/.cache/huggingface/datasets/d_bpedia14/dbpedia_14/2.0.0/7f0577ea0f4397b6b89bfe5c5f2c6b1b420990a1fc5e8538c7ab4ec40e46fa3e. Subsequent calls will reuse this data.


In [4]:
# dbpedia_dataset = dbpedia_dataset.rename_column("label", "labels") # snorkel uses "label"

In [5]:
def merge_title_with_content(example):
    example["content"] = example["title"] + " " + example["content"]
    return example

In [6]:
dbpedia_dataset = dbpedia_dataset.map(merge_title_with_content, num_proc=10)

In [134]:
dbpedia_dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 560000
    })
    test: Dataset({
        features: ['content', 'labels', 'title'],
        num_rows: 70000
    })
})

In [135]:
dbpedia_dataset['train']

Dataset({
    features: ['content', 'labels', 'title'],
    num_rows: 560000
})

In [136]:
data = {'labels': dbpedia_dataset['train']['labels'], # replacing with -1 because snorkel needs this for training set, for keyword creation use: dbpedia_dataset['train']['labels']
        'text': dbpedia_dataset['train']['content']} # need to replace "content" with "text" for snorkel to work

dbpedia_train_df = pd.DataFrame(data)
dbpedia_train_df['text'] = dbpedia_train_df['text'].str.lower()
dbpedia_train_df

,labels,text
0,0,e. d. abbott ltd abbott of farnham e d abbott limited was a british coachbuilding business based in farnham surrey trading under that name from 1929. a major part of their output was under sub-contract to motor vehicle manufacturers. their business closed in 1972.
1,0,schwan-stabilo schwan-stabilo is a german maker of pens for writing colouring and cosmetics as well as markers and highlighters for office use. it is the world's largest manufacturer of highlighter pens stabilo boss.
2,0,q-workshop q-workshop is a polish company located in poznań that specializes in designand production of polyhedral dice and dice accessories for use in various games (role-playing gamesboard games and tabletop wargames). they also run an online retail store and maintainan active forum community.q-workshop was established in 2001 by patryk strzelewicz – a student from poznań. initiallythe company sold its products via online auction services but in 2005 a website and online store wereestablished.
3,0,marvell software solutions israel marvell software solutions israel known as radlan computer communications limited before 2007 is a wholly owned subsidiary of marvell technology group that specializes in local area network (lan) technologies.
4,0,bergan mercy medical center bergan mercy medical center is a hospital located in omaha nebraska. it is part of the alegent health system.
...,...,...
559995,13,barking in essex barking in essex is a black comedy play directed by harry burton. it is based on the 2005 script by clive exton and made its world premiere at the wyndham's theatre in september 2013.
559996,13,science & spirit science & spirit is a discontinued american bimonthly magazine that covered scientific stories with an eye toward their spiritual implications. it was launched by the john templeton foundation in 1989 as a newsletter converted to a glossy magazine in 1998 then repositioned for a general readership in 2001. in 2003 it was purchased by heldref publications though the john templeton foundation continued to provide editorial support.
559997,13,the blithedale romance the blithedale romance (1852) is nathaniel hawthorne's third major romance. in hawthorne (1879) henry james called it the lightest the brightest the liveliest of hawthorne's unhumorous fictions.
559998,13,razadarit ayedawbon razadarit ayedawbon (burmese: ရာဇာဓိရာဇ် အရေးတော်ပုံ) is a burmese chronicle covering the history of ramanya from 1287 to 1421. the chronicle consists of accounts of court intrigues rebellions diplomatic missions wars etc. about half of the chronicle is devoted to the reign of king razadarit (r.


In [137]:
data = {'labels': dbpedia_dataset['test']['labels'],
        'text': dbpedia_dataset['test']['content']} # need to replace "content" with "text" for snorkel to work

dbpedia_test_df = pd.DataFrame(data)
del data
dbpedia_test_df['text'] = dbpedia_test_df['text'].str.lower()
dbpedia_test_df

,labels,text
0,0,ty ku ty ku /taɪkuː/ is an american alcoholic beverage company that specializes in sake and other spirits. the privately-held company was founded in 2004 and is headquartered in new york city new york. while based in new york ty ku's beverages are made in japan through a joint venture with two sake breweries. since 2011 ty ku's growth has extended its products into all 50 states.
1,0,odd lot entertainment oddlot entertainment founded in 2001 by longtime producers gigi pritzker and deborah del prete (the wedding planner) is a film production and financing company based in culver city california.oddlot produced the film version of orson scott card's sci-fi novel ender's game. a film version of this novel had been in the works in one form or another for more than a decade by the time of its release.
2,0,henkel henkel ag & company kgaa operates worldwide with leading brands and technologies in three business areas: laundry & home care beauty care and adhesive technologies. henkel is the name behind some of america’s favorite brands.
3,0,goat store the goat store (games of all type store) llc is one of the largest retro gaming online stores and an independent video game publishing label. additionally they are one of the primary sponsors for midwest gaming classic.
4,0,ragwing aircraft designs ragwing aircraft designs (also called the ragwing aeroplane company and ragwing aviation) was an american aircraft design and manufacturing company based in belton south carolina.
...,...,...
69995,13,energy victory energy victory: winning the war on terror by breaking free of oil is a 2007 book by robert zubrin. zubrin's central argument is that the decisive front in the war on terror is america's struggle for energy independence. he outlines the manner in which radical islam has been financed by oil revenues the technological feasibility of ethanol-fueled vehicles as well as the economic and agricultural imperatives for ethanol production and the environmental implications of his plan.
69996,13,bestiario bestiario is a book of 8 short stories written by julio cortázar.
69997,13,wuthering heights wuthering heights is a novel by emily brontë written between october 1845 and june 1846 and published in 1847 under the pseudonym ellis bell. it was her first and only published novel: she died the following year at age 30. the decision to publish came after the success of her sister charlotte's novel jane eyre.
69998,13,l'indépendant l'indépendant is a newspaper published in luxembourg from 1945.


## A gentle introduction to LFs

Labeling functions (LFs) help users encode domain knowledge and other supervision sources programmatically.

LFs are heuristics that take as input a data point and either assign a label to it (in this case, HAM or SPAM) or abstain (don’t assign any label). Labeling functions can be noisy: they don’t have perfect accuracy and don’t have to label every data point. Moreover, different labeling functions can overlap (label the same data point) and even conflict (assign different labels to the same data point). This is expected, and we demonstrate how we deal with this later.

Because their only requirement is that they map a data point a label (or abstain), they can wrap a wide variety of forms of supervision. Examples include, but are not limited to:

    Keyword searches: looking for specific words in a sentence
    Pattern matching: looking for specific syntactical patterns
    Third-party models: using an pre-trained model (usually a model for a different task than the one at hand)
    Distant supervision: using external knowledge base
    Crowdworker labels: treating each crowdworker as a black-box function that assigns labels to subsets of the data

### a) Exploring the training set for initial ideas

We’ll start by looking at 20 random data points from the train set to generate some ideas for LFs.


In [138]:
ABSTAIN = -1
Company = 0
EducationalInstitution = 1
Artist = 2
Athlete = 3
OfficeHolder = 4
MeanOfTransportation = 5
Building = 6
NaturalPlace = 7
Village = 8
Animal = 9
Plant = 10
Album = 11
Film = 12
WrittenWork = 13



In [139]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 200)

In [182]:
# # for keyword creation
df_view = dbpedia_train_df[dbpedia_train_df['labels'] == 13].sample(1000, random_state=42)
df_view

,labels,text
552823,13,il piccolo il piccolo is the main daily newspaper of trieste italy. it has been founded in 1881 and it has since kept a moderately progressive political stance. local editions for the towns of monfalcone gorizia and istria (croatia) are also printed. the name il piccolo derives from the paper's original small format.
536298,13,drakas! drakas! is an anthology in s.m. stirling's alternate history series the domination.
548505,13,neural plasticity (journal) neural plasticity is a peer-reviewed scientific journal covering all aspects of neuroplasticity especially when concerning its functional involvement in the regulation of behavior and in psychopathology. the journal was established in 1989 as the journal of neural transplantation and renamed in 1991 to journal of neural transplantation and plasticity before obtaining its current name in 1998. it is published by hindawi publishing corporation.
526689,13,quentins quentins is a 2002 novel by irish author maeve binchy.
546893,13,new york world the new york world was a newspaper published in new york city from 1860 until 1931. the paper played a major role in the history of american newspapers. it was a leading national voice of the democratic party. from 1883 to 1911 under publisher joseph pulitzer it became a pioneer in yellow journalism capturing readers' attention and pushing its daily circulation to the one-million mark.
...,...,...
528897,13,the long war (novel) the long war is a science fiction novel by terry pratchett and stephen baxter. it is a sequel to their award-winning parallel-earth novel the long earth. this book is the second in a projected five-book series.
528957,13,ranger rick ranger rick originally ranger rick's nature magazine is a children’s nature magazine that is published by the national wildlife federation. the magazine offers activities for children ages 7 and up in order to spark their interest in the outdoors and become more actively involved in the environment. the magazine's primary intention is to instill a passion for nature and promote activity outdoors.
539003,13,ultimate comics: x ultimate comics: x is a limited series from marvel comics written by jeph loeb with pencils by art adams originally planned as an ongoing bi-monthly series.the story picks up after the events in ultimatum and serves as a quasi-sequel/spin-off of ultimate x-men. originally scheduled to be published bi-monthly alternating months with loeb's other ultimate title ultimate comics: new ultimates x suffered from long repeated delays and became instead a prologue to ultimate comics: x-men.
546394,13,mitsudomoe (manga) mitsudomoe (みつどもえ lit. three way struggle) is a gag manga series by manga creator norio sakurai about the adventures of the marui triplets in sixth grade and their newly hired teacher satoshi yabe as he falls in love with the school nurse. it was serialized in akita shoten shōnen magazine weekly shōnen champion since 2006 before moving to bessatsu shōnen champion in 2012. the serial chapters collected into twelve tankōbon.


In [183]:
# Showing the most common words to make the keyword creation process quicker
# We are choosing some of words in this list to populate the label:keyword dictionary

# from collections import Counter
# Counter(" ".join(df_view["content"]).split()).most_common(100) # slower than pd.Series()

# faster
pd.Series(' '.join(df_view.text).lower().split()).value_counts()[:100]

the              3843
of               1931
in               1739
a                1664
and              1644
is               1527
by               1332
it                631
was               629
published         607
to                561
novel             425
book              414
on                342
as                332
an                300
first             281
for               274
with              241
journal           240
written           229
from              203
series            202
magazine          184
that              178
his               175
story             155
newspaper         140
has               137
american          125
its               123
fiction           119
which             117
who               117
science           116
new               105
stories            99
at                 99
are                96
author             96
&                  90
been               89
short              83
about              81
world              80
manga     

## Creating Label:Keyword Dictionary

In [189]:
import spacy
from spacy.matcher import PhraseMatcher

# Creating keyword lists with string and split(', ').

# keywords_list = ["""company, headquarter, corporate, finance, ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, inc., foundation, newspaper""".split(', '),
#                  """university, students, bachelor, degree, school, academy, college, high, located, public, city, academy, county, institute, national, research, business, grammar, government, technology, medicine, mascot, academic, board, program, co-educational, economics, junior, science, schools, faculty""".split(', '),
#                  """dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, english, american, chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, screenwriter, prose, poet, career, jazz, folk""".split(', '),
#                  """rugby, player, football, professional, nfl, league, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, mlb, pitcher, fencer, driver, american, canadian, english, belgian, attended, season, champion, motorcycle""".split(', '),
#                  """born, author, commentator, director, professor, leader, governor, politician, minister, president, died, representatives, assembly, republic, democratic, house of representatives, house, election, member of parliament, party, speaker, president, lawyer, liberal, candidate, election, deputy, prime minister, official""".split(', '),
#                  """ship, aircraft, boeing, navy, destroyer, diesel, rail, warship, transport, submarine, trike, aerobatic, motorcar, railway, monoplane, vessel, motorcycle, navigation, railway, cars, airliner, naval, whaleship, rail, automobile, flown, ferry, wing, tailplane, car, convoy, flown, naval, locomotive, vehicle, flagship, cruise ship, boat, convoy, automaker, battleships""".split(', '),
#                  """church, church, center, house, historic, dam, district, district, castle, hospital, institution, museum, victorian, farm, building, mall, restaurant, shopping mall, centre, supermarket, built in""".split(', '),
#                  """river, mountain, hill, hills, land, lake, km, m, forest, creek, ocean, stream, strait, gulf, peak, elevation, glacier, volcanic, corona, tributary, mount, flows, border, watershed""".split(', '),
#                  """rural, population, census, central, province, state, village, district, eastern, western, kilometres, km, mi, county, south, north, south-west, south-east, north-west, north-east, town, border, regional, capital, municipality, block, located, families, administrative, federation, india, united states, turkey, iran""".split(', '),
#                  """habitat, species, beetles, subfamily, snail, endemic, family, extinct, shelled, squids, octopuses, wingspan, moth, frogs, dry land, humidity, fruit flies, genus, horse, racehorse, thoroughbred, farm, farms, frog, butterfly, class, flatworm, fish, bred, shark, tropical, subtropical, sphinx, dog, cat, mouse, lion, jaguar, sea, habitats, subclass, populations, fossil""".split(', '),
#                  """plant, family, species, vegetative subspecies, dipterocarpaceae, tillandsia, genus, endemic, orchid, daisy, flower, flowering, plants, legume, habitat, green alga, lettuce, kelp, gutweed, succulent, microphylia, ulmus, coffee, soil, tropical, forest, wood, leaf, cultivated, tree, trees, aster, algae, sedge, grows, evergreen, fruit, seed, seeds, herbs, herb, bulrush, subtropical, violet, floral, meliaceae, wild, grass""".split(', '),
#                  """album, country, singer, band, vocalist, member, guitarist, debut, studio, metal, records, produced, songs, live at, performance, indie, folk, musician, released, ep, music, rapper, official, cd, label, tracks, remastered, reissued, pop, release, recorded, producer, full-length, bonus, chart, reunited, grammy, billboard, featured, concert, singer-songwriter, songs""".split(', '),
#                  """film, directed, starring, drama, based, released, novel, comedy, american, stars, produced, international, cinema, festival, documentary, biographical, love, romance, comedy-romance, movie""".split(', '),
#                  """published, book, novel, journal, series, written, story, magazine, newspaper, daily, stories, peer-reviewed, fiction, covers, comic, volume, science, fantasy, edition, writer, law, created, research, established, history, weekly, issue, travel, academic, mystery, media, author, work, god""".split(', ')]

keywords_list = ["""company, headquarter, corporate, finance, ltd., airline, firm, commerce, manufacturer, factory, based in, based out of, founded, corporation, inc., foundation, newspaper, services, service, products, group, bank, game, business, management, independent, owned, software, operates, limited, technology, brand, offices, million, financial, public, commercial, publishing, media, companies, operated, systems, mobile, games""".split(', '),
                 """university, students, bachelor, degree, school, academy, college, high, located, public, city, academy, county, institute, national, research, business, grammar, government, technology, medicine, mascot, academic, board, program, co-educational, economics, junior, science, schools, faculty, education, private, engineering, catholic, community, christian, medical, elementary, law, higher, founded""".split(', '),
                 """dancer, writer, artist, actor, singer-songwriter, teacher, scholar, professor, composer, actress, pianist, novelist, singer, songwriter, born, English, American, Chinese, guitarist, author, drummer, vocalist, saxophonist, painter, Canadian, member of, film, screenwriter, prose, poet, career, jazz, folk, member, work, album, producer, released, award, career, painter, band, musician, rock, album, author, university, television, school, art, released, composer""".split(', '),
                 """rugby, player, football, professional, nfl, league, injury, 1st round, contract, footballer, wrestling, lineman, cricketer, born, retired, former, mlb, pitcher, fencer, driver, american, canadian, english, attended, season, champion, motorcycle, (born, played, world, playing, major, plays, national, team, won, baseball, hockey, cricket, career, club, rugby, league., australian, olympics, seasons, championship, team., medal, ice, cup, round, player.""".split(', '),
                 """born, author, commentator, director, professor, leader, governor, politician, minister, president, died, representatives, assembly, republic, democratic, house of representatives, house, election, member of parliament, party, speaker, president, lawyer, liberal, candidate, election, deputy, prime minister, official, served, district, elected, district, republican, county, general, university, mayor, government, legislative, secretary, serving, political, appointed, represented, national, politician., prime, lawyer, senate, former""".split(', '),
                 """ship, aircraft, boeing, navy, destroyer, diesel, rail, warship, transport, submarine, trike, aerobatic, motorcar, railway, monoplane, vessel, motorcycle, navigation, railway, cars, airliner, naval, whaleship, rail, automobile, flown, ferry, wing, tailplane, car, convoy, flown, naval, locomotive, vehicle, flagship, cruise ship, boat, convoy, automaker, battleships, built, hms, royal, world, service, designed, commissioned, company, produced, laid, launched, ss, operated, design, sponsored, class, uss, war""".split(', '),
                 """church, church, center, house, historic, dam, district, district, castle, hospital, institution, museum, victorian, farm, building, mall, restaurant, shopping mall, centre, supermarket, built in, located, national, register, places, county, listed, museum, street, city, hospital, hotel, centre, designed, school, home, park, brick, mall, art, tower, revival, complex, village, parish, castle, main, features, architect, cathedral, road, farm, structure, office, downtown, area, frame, constructed, style, hall, site""".split(', '),
                 """river, mountain, hill, hills, land, lake, km, m, forest, forests, creek, ocean, stream, strait, gulf, peak, elevation, glacier, volcanic, corona, tributary, mount, flows, border, watershed, located, county, range, peak, mountains, part, highest, crater, lies, national, elevation, sea, water, park, reservoir, high, situated, bay, river.""".split(', '),
                 """rural, population, census, central, province, state, village, district, eastern, western, kilometres, km, mi, county, south, north, south-west, south-east, north-west, north-east, town, border, regional, capital, municipality, block, located, families, administrative, federation, india, united states, turkey, iran, mi), families., approximately, history, region, town, area, romanized, gmina""".split(', '),
                 """habitat, species, forest, forests, beetles, subfamily, snail, endemic, family, extinct, shelled, squids, octopuses, wingspan, moth, frogs, dry land, humidity, fruit flies, genus, horse, racehorse, thoroughbred, farm, farms, frog, butterfly, class, flatworm, fish, bred, shark, tropical, subtropical, sphinx, dog, cat, mouse, lion, jaguar, sea, habitats, subclass, populations, fossil, found, gastropod, described, mollusk, marine, larvae, native, arctiidae, moist, bird, notiobia, feed""".split(', '),
                 """plant, family, species, vegetative subspecies, dipterocarpaceae, tillandsia, genus, endemic, orchid, daisy, flower, flowering, plants, legume, habitat, green alga, lettuce, kelp, gutweed, succulent, microphylia, ulmus, coffee, soil, tropical, forest, wood, leaf, cultivated, tree, trees, aster, algae, sedge, grows, evergreen, fruit, seed, seeds, herbs, herb, bulrush, subtropical, violet, floral, meliaceae, wild, grass, native, known, found, leaves, grows, plants, flowers, growing, forests""".split(', '),
                 """album, country, singer, band, vocalist, member, guitarist, debut, studio, metal, records, produced, songs, live at, performance, indie, folk, musician, released, ep, music, rapper, official, cd, label, tracks, remastered, reissued, pop, release, recorded, producer, full-length, bonus, chart, reunited, grammy, billboard, featured, concert, singer-songwriter, songs, live, recorded, rock, album), records., release, compilation, single, label, cover""".split(', '),
                 """film, directed, starring, drama, based, released, novel, comedy, american, stars, produced, international, cinema, festival, documentary, biographical, love, romance, comedy-romance, movie, director, roles., series, life, award, music, screenplay, romantic, thriller, shot, action, horror, (film), written, based, silent""".split(', '),
                 """published, book, novel, journal, series, written, story, magazine, newspaper, daily, stories, peer-reviewed, fiction, covers, comic, volume, science, fantasy, edition, writer, law, created, research, established, history, weekly, issue, travel, academic, mystery, media, author, work, god, manga, world, (novel), illustrated, award, news, fantasy, books, collection, life, publication, edition, based, established""".split(', ')]


labels_list = ['Company', 'EducationalInstitution', 'Artist', 'Athlete', 'OfficeHolder',
               'MeanOfTransportation','Building', 'NaturalPlace', 'Village',
               'Animal', 'Plant', 'Album', 'Film', 'WrittenWork']

label_keyword_dict = dict(zip(labels_list, keywords_list))
print(label_keyword_dict['NaturalPlace'])

nlp = spacy.load("en_core_web_sm")
matcher = PhraseMatcher(nlp.vocab)

['river', 'mountain', 'hill', 'hills', 'land', 'lake', 'km', 'm', 'forest', 'forests', 'creek', 'ocean', 'stream', 'strait', 'gulf', 'peak', 'elevation', 'glacier', 'volcanic', 'corona', 'tributary', 'mount', 'flows', 'border', 'watershed', 'located', 'county', 'range', 'peak', 'mountains', 'part', 'highest', 'crater', 'lies', 'national', 'elevation', 'sea', 'water', 'park', 'reservoir', 'high', 'situated', 'bay', 'river.']


In [190]:
# Running nlp.make_doc to speed things up
for label in label_keyword_dict:
  patterns = [nlp.make_doc(text) for text in label_keyword_dict[label]]
  matcher.add(label, patterns)

In [191]:
doc = nlp("monika bohge monika bohge (lüdenscheid 1947) is a german writer and actor.")
matches = matcher(doc)
for match_id, start, end in matches:
    rule_id = nlp.vocab.strings[match_id]
    span = doc[start:end]
    print(rule_id, span.text)

WrittenWork writer
Artist writer
Artist actor


In [192]:
matches[0]

(10780820017166675169, 11, 12)

## Snorkel with Automated Labeling Functions

In [ ]:
# labeling function for all labels
@nlp_labeling_function() # labeling function for using spaCy
def lf_labeler(x, label_keyword_dict):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    for match_id in matches:
        labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
    if labels: # runs if labels is not empty
        return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

In [59]:
x = "monika bohge monika bohge (lüdenscheid 1947) is a german writer and actor."

In [89]:
class all_labeling_functions(object):
    def __init__(self, labels):
        for label in labels:
            setattr(self, label, functools.partial(self._myfunction, label))
    
    @nlp_labeling_function() # labeling function for using spaCy
    def lf_labeler(x, label_keyword_dict=label_keyword_dict):
        labels = []
        doc = nlp(x) # tokenizing the input text
        matches = PhraseMatcher(doc) # finding all the token words that match a specific label
        
        for match_id in matches:
          labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
        
        if len(labels) != 0: # runs if labels is not empty
          return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
        
        else:
          return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

lfs = []

for i in range(len(labels_list)):
  lf_labeler = all_labeling_functions(i)
  # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

  lfs.append(all_labeling_functions(labels_list))

# myfunction = all_labeling_functions(labels_list)
# print(myfunction)

In [90]:
lfs

In [132]:
from types import FunctionType
from copy import copy

lf_list = []

@nlp_labeling_function() # labeling function for using spaCy
def lf_labeler(x):
    labels = []
    doc = nlp(x) # tokenizing the input text
    matches = PhraseMatcher(doc) # finding all the token words that match a specific label
    
    for match_id in matches:
      labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
    
    if len(labels) != 0: # runs if labels is not empty
      return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    
    else:
      return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

  # namespace = sys._getframe(0).f_globals
  # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

def copy_function(fn, name):

    return FunctionType(name=name)

for label in labels_list:
    name = 'lf_labeler_' + str(label)
    lf_list.append(copy_function(lf_labeler, name))

lf_list

TypeError: ignored

In [112]:
from snorkel.labeling.lf.nlp import nlp_labeling_function
from functools import partial
import sys

namespace = sys._getframe(0).f_globals

def all_labeling_functions(label): # labeling function for all labels

  @nlp_labeling_function() # labeling function for using spaCy
  def lf_labeler(x):
      labels = []
      doc = nlp(x) # tokenizing the input text
      matches = PhraseMatcher(doc) # finding all the token words that match a specific label
      
      for match_id in matches:
        labels.append(nlp.vocab.strings[match_id]) # adds every label where there was a match
      
      if len(labels) != 0: # runs if labels is not empty
        return labels # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
      
      else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

    # namespace = sys._getframe(0).f_globals
    # namespace['lf_labeler_%s'%i] = functools.partial(all_labeling_functions, i)

  return lf_labeler

lfs = []

lf_labeler_list = {f'lf_labeler_{k}': partial(lf_labeler, i=k) for k in range(len(labels_list))}
for i in range(len(labels_list)):
  lfs.append(lf_labeler_list[f'lf_labeler_{i}'])


# lf_labeler_list = {f'lf_labeler_{k}': partial(lf_labeler, i=k) for k in range(len(labels_list))}
for label in labels_list:
  lfs[f'lf_labeler_{label}'] = all_labeling_functions(label)

SyntaxError: ignored

In [110]:
lf_labeler_list

{'lf_labeler_0': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_1': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_2': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_3': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_4': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_5': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_6': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []],
 'lf_labeler_7': NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []]}

In [106]:
lfs

[functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=0),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=1),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=2),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=3),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=4),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=5),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=6),
 functools.partial(NLPLabelingFunction lf_labeler, Preprocessors: [SpacyPreprocessor SpacyPreprocessor, Pre: []], i=7)]

In [ ]:
# from snorkel.labeling import LabelingFunction


# def keyword_lookup(x, keywords, label):
#     if any(word in x.text.lower() for word in keywords):
#         return label
#     return ABSTAIN


# def make_keyword_lf(keywords, label=SPAM):
#     return LabelingFunction(
#         name=f"keyword_{keywords[0]}",
#         f=keyword_lookup,
#         resources=dict(keywords=keywords, label=label),
#     )

## Snorkel with Manual Labeling Functions

### Matcher Approach

In [93]:
from snorkel.labeling.lf.nlp import nlp_labeling_function

@nlp_labeling_function()
def lf_company(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Company' for label_id in label_ids]):
        return Company # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_educational_institution(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'EducationalInstitution' for label_id in label_ids]):
        return EducationalInstitution # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_artist(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Artist' for label_id in label_ids]):
        return Artist # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_athlete(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Athlete' for label_id in label_ids]):
        return Athlete # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_office_holder(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'OfficeHolder' for label_id in label_ids]):
        return OfficeHolder # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_mean_of_transportation(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'MeanOfTransportation' for label_id in label_ids]):
        return MeanOfTransportation # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_building(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Building' for label_id in label_ids]):
        return Building # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_natural_place(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'NaturalPlace' for label_id in label_ids]):
        return NaturalPlace # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_village(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Village' for label_id in label_ids]):
        return Village # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_animal(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Animal' for label_id in label_ids]):
        return Animal # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_plant(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Plant' for label_id in label_ids]):
        return Plant # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match


@nlp_labeling_function()
def lf_album(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Album' for label_id in label_ids]):
        return Album # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_film(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'Film' for label_id in label_ids]):
        return Film # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

@nlp_labeling_function()
def lf_written_work(x):
    doc = nlp(str(x)) # tokenizing the input text
    matches = matcher(doc) # finding all the token words that match a specific label
    label_ids = [item[0] for item in matches]
    if any([nlp.vocab.strings[label_id] == 'WrittenWork' for label_id in label_ids]):
        return WrittenWork # this would return all the labels that were matches in the text document. Not sure if it's possible to return the list of labels like this.
    else:
        return ABSTAIN # abstains from using this text example in the dataset creation because there was no match

### Keyword Lookup Approach

In [186]:
Company_keywords = label_keyword_dict['Company']
EducationalInstitution_keywords = label_keyword_dict['EducationalInstitution']
Artist_keywords = label_keyword_dict['Artist']
Athlete_keywords = label_keyword_dict['Athlete']
OfficeHolder_keywords = label_keyword_dict['OfficeHolder']
MeanOfTransportation_keywords = label_keyword_dict['MeanOfTransportation']
Building_keywords = label_keyword_dict['Building']
NaturalPlace_keywords = label_keyword_dict['NaturalPlace']
Village_keywords = label_keyword_dict['Village']
Animal_keywords = label_keyword_dict['Animal']
Plant_keywords = label_keyword_dict['Plant']
Album_keywords = label_keyword_dict['Album']
Film_keywords = label_keyword_dict['Film']
WrittenWork_keywords = label_keyword_dict['WrittenWork']

company_keywords

['published',
 'book',
 'novel',
 'journal',
 'series',
 'written',
 'story',
 'magazine',
 'newspaper',
 'daily',
 'stories',
 'peer-reviewed',
 'fiction',
 'covers',
 'comic',
 'volume',
 'science',
 'fantasy',
 'edition',
 'writer',
 'law',
 'created',
 'research',
 'established',
 'history',
 'weekly',
 'issue',
 'travel',
 'academic',
 'mystery',
 'media',
 'author',
 'work',
 'god']

In [193]:
keywords_list[0]

['company',
 'headquarter',
 'corporate',
 'finance',
 'ltd.',
 'airline',
 'firm',
 'commerce',
 'manufacturer',
 'factory',
 'based in',
 'based out of',
 'founded',
 'corporation',
 'inc.',
 'foundation',
 'newspaper',
 'services',
 'service',
 'products',
 'group',
 'bank',
 'game',
 'business',
 'management',
 'independent',
 'owned',
 'software',
 'operates',
 'limited',
 'technology',
 'brand',
 'offices',
 'million',
 'financial',
 'public',
 'commercial',
 'publishing',
 'media',
 'companies',
 'operated',
 'systems',
 'mobile',
 'games']

In [196]:
from snorkel.labeling.lf.nlp import nlp_labeling_function

def keyword_lookup(x, keywords, label):
    tokens = ''.join([token for token in x.doc])
    if any(word in tokens for word in keywords):
        return label
    return ABSTAIN


@nlp_labeling_function()
def lf_company(x):
  return keyword_lookup(x, keywords_list[0], labels_list[0])

@nlp_labeling_function()
def lf_educational_institution(x):
  return keyword_lookup(x, keywords_list[1], labels_list[1])

@nlp_labeling_function()
def lf_artist(x):
  return keyword_lookup(x, keywords_list[2], labels_list[2])

@nlp_labeling_function()
def lf_athlete(x):
  return keyword_lookup(x, keywords_list[3], labels_list[3])

@nlp_labeling_function()
def lf_office_holder(x):
  return keyword_lookup(x, keywords_list[4], labels_list[4])

@nlp_labeling_function()
def lf_mean_of_transportation(x):
  return keyword_lookup(x, keywords_list[5], labels_list[5])

@nlp_labeling_function()
def lf_building(x):
  return keyword_lookup(x, keywords_list[6], labels_list[6])

@nlp_labeling_function()
def lf_natural_place(x):
  return keyword_lookup(x, keywords_list[7], labels_list[7])

@nlp_labeling_function()
def lf_village(x):
  return keyword_lookup(x, keywords_list[8], labels_list[8])

@nlp_labeling_function()
def lf_animal(x):
  return keyword_lookup(x, keywords_list[9], labels_list[9])

@nlp_labeling_function()
def lf_plant(x):
  return keyword_lookup(x, keywords_list[10], labels_list[10])

@nlp_labeling_function()
def lf_album(x):
  return keyword_lookup(x, keywords_list[11], labels_list[11])

@nlp_labeling_function()
def lf_film(x):
  return keyword_lookup(x, keywords_list[12], labels_list[12])

@nlp_labeling_function()
def lf_written_work(x):
  return keyword_lookup(x, keywords_list[13], labels_list[13])


In [197]:
lfs = [
       lf_company,
       lf_educational_institution,
       lf_artist,
       lf_athlete,
       lf_office_holder,
       lf_mean_of_transportation,
       lf_building,
       lf_natural_place,
       lf_village,
       lf_animal,
       lf_plant,
       lf_album,
       lf_film,
       lf_written_work
]

## Applying Snorkel Labeling Functions

In [198]:
# company_dataset = dbpedia_train_df.iloc[0:1000]
# company_dataset

df_small_train_dbpedia = dbpedia_train_df.sample(1000, random_state=42).reset_index(drop=True)
df_small_test_dbpedia = dbpedia_test_df.sample(1000, random_state=42).reset_index(drop=True)
df_small_train_dbpedia

,labels,text
0,0,sterling piano company the sterling piano company was a piano manufacturer in derby connecticut. the company was founded in 1873 by charles a. sterling as the sterling organ company. sterling had purchased the birmingham organ company in 1871 and had $30000 to fund the company. the sterling organ company began making pianos in 1885.
1,5,nyc s-motor s-motor was the class designation given by the new york central to its alco-ge built s-1 s-2 s-2a and s-3 electric locomotives. the s-motors hold the distinction of being the world's first mass-produced main line electric locomotives with the prototype #6000 being constructed in 1904. the s-motors would serve alone until the more powerful t-motors began to arrive in 1913 eventually displacing them from main line passenger duties.
2,2,axel zwingenberger axel zwingenberger (born may 7 1955 hamburg germany) is a blues and boogie-woogie pianist and songwriter. he is considered one of the finest boogie-woogie music masters in the world.
3,9,sceptrophasma hispidulum sceptrophasma hispidulum commonly known as the andaman islands stick insect is a species of the stick insect family. it originates from the andaman islands and is commonly found in tropical forests there. they eat a variety of foliage though in captivity they commonly eat blackberry bramble hawthorn oak rose and lettuce. the species has the phasmid study group number psg183.
4,7,nucet river (chiojdeanca) the nucet river is a tributary of the chiojdeanca river in romania.
...,...,...
995,2,pat arrowsmith pat arrowsmith (born march 2 1930) is a british author and peace campaigner.arrowsmith was educated at cheltenham ladies college read history at the university of cambridge and then read social science at the university of liverpool and at ohio university as a us-uk fulbright scholar. she is a co-founder of the campaign for nuclear disarmament.she has served eleven prison sentences for her political activities.
996,10,cleistanthus cleistanthus is a plant genus of the family phyllanthaceae. the genus comprises 140 species found from africa to the pacific islands. cleistanthus collinus is known for being toxic and frequently used for homicidal or suicidal purposes.
997,13,kyunghyang shinmun the kyunghyang shinmun or kyonghyang sinmun is a major daily newspaper published in south korea. it is based in seoul. the name literally means urbi et orbi daily news.
998,10,cyathea atropurpurea cyathea atropurpurea is a species of tree fern native to the islands of luzon mindanao leyte and mindanao in the philippines where it grows in forest at above 1000 m. the erect trunk is slender and may be up to 3 m tall. fronds are bipinnate and 1-2 m long. characteristically of this species the final pair of pinnae are usually reduced and occur towards the base of the stipe. these along with the stipe bases are persistent and retained around the trunk long after withering.


In [96]:
from snorkel.labeling import PandasLFApplier
applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_small_train_dbpedia)
L_test = applier.apply(df=df_small_test_dbpedia)

Streaming output truncated to the last 5000 lines.

 29%|██▊       | 286/1000 [08:02<20:29,  1.72s/it]






 29%|██▊       | 287/1000 [08:04<20:02,  1.69s/it]






 29%|██▉       | 288/1000 [08:05<20:36,  1.74s/it]






 29%|██▉       | 289/1000 [08:07<19:57,  1.68s/it]






 29%|██▉       | 290/1000 [08:09<19:25,  1.64s/it]






 29%|██▉       | 291/1000 [08:10<19:20,  1.64s/it]






 29%|██▉       | 292/1000 [08:12<18:33,  1.57s/it]






 29%|██▉       | 293/1000 [08:13<18:06,  1.54s/it]






 29%|██▉       | 294/1000 [08:15<18:32,  1.58s/it]






 30%|██▉       | 295/1000 [08:16<18:41,  1.59s/it]






 30%|██▉       | 296/1000 [08:18<18:31,  1.58s/it]






 30%|██▉       | 297/1000 [08:20<18:52,  1.61s/it]






 30%|██▉       | 298/1000 [08:21<19:43,  1.69s/it]






 30%|██▉       | 299/1000 [08:23<19:05,  1.63s/it]






 30%|███       | 300/1000 [08:24<18:36,  1.60s/it]






 30%|███       | 301/1000 [08:26<19:03,  1.64s/it]






 30%|███       | 302/1000 [08:28<19:

In [100]:
L_train

array([[ 0, -1, -1, ..., -1, -1, -1],
       [-1, -1, -1, ..., 11, 12, -1],
       [-1, -1,  2, ..., 11, -1, -1],
       ...,
       [ 0, -1, -1, ..., -1, 12, 13],
       [-1, -1, -1, ..., -1, -1, -1],
       [-1, -1,  2, ..., -1, 12, 13]])

In [101]:
type(L_train)

numpy.ndarray

In [104]:
import pickle

with open('L_train.pkl','wb') as f:
  pickle.dump(L_train, f)
with open('L_train.pkl','rb') as f:
  x = pickle.load(f)
  print(x.shape)

(1000, 14)


In [105]:
with open('L_test.pkl','wb') as f:
  pickle.dump(L_test, f)
with open('L_test.pkl','rb') as f:
  x = pickle.load(f)
  print(x.shape)

(1000, 14)


In [98]:
from snorkel.labeling import LFAnalysis
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
lf_company,0,[0],0.112,0.108,0.108
lf_educational_institution,1,[1],0.378,0.372,0.372
lf_artist,2,[2],0.335,0.334,0.334
lf_athlete,3,[3],0.320,0.317,0.317
lf_office_holder,4,[4],0.251,0.249,0.249
lf_mean_of_transportation,5,[5],0.066,0.057,0.057
lf_building,6,[6],0.212,0.211,0.211
lf_natural_place,7,[7],0.136,0.117,0.117
lf_village,8,[8],0.390,0.383,0.383
lf_animal,9,[9],0.193,0.192,0.192


In [106]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=14, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=123)

In [110]:
Y_test = df_small_test_dbpedia.labels.values
LFAnalysis(L_test, lfs).lf_summary(Y_test)

,j,Polarity,Coverage,Overlaps,Conflicts,Correct,Incorrect,Emp. Acc.
lf_company,0,[0],0.110,0.103,0.103,51,59,0.463636
lf_educational_institution,1,[1],0.368,0.361,0.361,71,297,0.192935
lf_artist,2,[2],0.290,0.286,0.286,66,224,0.227586
lf_athlete,3,[3],0.256,0.255,0.255,68,188,0.265625
lf_office_holder,4,[4],0.218,0.212,0.212,63,155,0.288991
lf_mean_of_transportation,5,[5],0.088,0.083,0.083,65,23,0.738636
lf_building,6,[6],0.230,0.225,0.225,83,147,0.360870
lf_natural_place,7,[7],0.152,0.139,0.139,62,90,0.407895
lf_village,8,[8],0.405,0.395,0.395,69,336,0.170370
lf_animal,9,[9],0.209,0.207,0.207,70,139,0.334928


In [111]:
label_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Majority Vote Accuracy:':<25} {label_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=Y_test, tie_break_policy="random")[
    "accuracy"
]
print(f"{'Label Model Accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority Vote Accuracy:   46.2%
Label Model Accuracy:     46.2%


## Filtering out unlabeled data points

As we saw earlier, some of the data points in our train set received no labels from any of our LFs. These data points convey no supervision signal and tend to hurt performance, so we filter them out before training using a built-in utility.

In [114]:
probs_train = label_model.predict_proba(L_train)
probs_train

array([[4.52733310e-01, 1.09449255e-01, 1.76981234e-02, ...,
        3.46087310e-02, 1.17073017e-01, 9.79146147e-02],
       [1.28138792e-01, 3.94428720e-04, 1.00144036e-01, ...,
        2.54975012e-01, 4.19615135e-03, 3.61581578e-02],
       [7.96311597e-05, 6.82134611e-04, 4.13196281e-01, ...,
        4.85173945e-02, 3.89070362e-02, 3.33404765e-03],
       ...,
       [3.72906810e-01, 1.20257606e-01, 8.46017360e-03, ...,
        2.23398397e-02, 1.26739285e-01, 3.08053323e-01],
       [1.12634277e-03, 7.68121763e-07, 9.87970092e-05, ...,
        4.62459227e-06, 9.52367601e-09, 2.59774852e-05],
       [1.78858109e-04, 2.17281014e-03, 4.57318604e-01, ...,
        2.46892531e-02, 7.90342664e-02, 2.46838993e-02]])

In [115]:
from snorkel.labeling import filter_unlabeled_dataframe

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(
    X=df_small_train_dbpedia, y=probs_train, L=L_train
)

In [123]:
df_small_train_dbpedia['labels'] = label_model.predict(L_train)

In [128]:
df_small_train_dbpedia.to_csv('./df_small_train_dbpedia.csv')

In [116]:
!pip install transformers==4.5.1 --quiet
!pip install pytorch_lightning==1.2.10 --quiet
!pip install wandb --quiet

     |████████████████████████████████| 2.1MB 10.0MB/s 
     |████████████████████████████████| 901kB 59.0MB/s 
     |████████████████████████████████| 3.3MB 48.1MB/s 
     |████████████████████████████████| 849kB 11.2MB/s 
     |████████████████████████████████| 184kB 52.5MB/s 
     |████████████████████████████████| 276kB 43.5MB/s 
     |████████████████████████████████| 10.6MB 21.6MB/s 
     |████████████████████████████████| 829kB 72.4MB/s 
     |████████████████████████████████| 1.3MB 57.9MB/s 
     |████████████████████████████████| 296kB 62.0MB/s 
     |████████████████████████████████| 143kB 62.7MB/s 
ERROR: snorkel 0.9.7 has requirement tensorboard<2.0.0,>=1.14.0, but you'll have tensorboard 2.4.1 which is incompatible.
     |████████████████████████████████| 1.8MB 11.7MB/s 
     |████████████████████████████████| 102kB 8.1MB/s 
     |████████████████████████████████| 133kB 60.2MB/s 
     |████████████████████████████████| 163kB 43.8MB/s 
     |████████████████████████████████

In [118]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, AdamW, get_linear_schedule_with_warmup, AutoModelForSequenceClassification, AutoTokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional.classification import auroc
from datasets import load_dataset

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt

from tqdm import tqdm
import wandb

In [119]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

RANDOM_SEED = 42
BASE_MODEL_NAME = 'bert-base-cased'

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [120]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [121]:
%env WANDB_LOG_MODEL=true

env: WANDB_LOG_MODEL=true


In [133]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)

In [ ]:
class dbpediaDataset(Dataset):

  def __init__(
      self,
      data: pd.DataFrame,
      tokenizer: tokenizer,
      text_max_token_len: int = 512
  ):

    self.tokenizer = tokenizer
    self.data = data
    self.text_max_token_len = text_max_token_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    text = data_row['text']

    text_encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="np",
    )

In [131]:
def encode(batch):
    return tokenizer(
        batch,
        add_special_tokens=True,
        max_length=512,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        return_tensors="np",
    )

In [132]:
df_small_train_dbpedia_torch = df_small_train_dbpedia['text'].map(encode, batched=True, num_proc=10)
df_small_train_dbpedia_torch.set_format(type="torch", 
                           columns=["input_ids", "token_type_ids", 
                                    "attention_mask", "labels"])

TypeError: ignored

In [ ]:
from snorkel.classification import DictDataLoader
from model import SceneGraphDataset, create_model

df_train["labels"] = label_model.predict(L_train)

if sample:
    TRAIN_DIR = "data/VRD/sg_dataset/samples"
else:
    TRAIN_DIR = "data/VRD/sg_dataset/sg_train_images"

dl_train = DictDataLoader(
    SceneGraphDataset("train_dataset", "train", TRAIN_DIR, df_train),
    batch_size=16,
    shuffle=True,
)

dl_valid = DictDataLoader(
    SceneGraphDataset("valid_dataset", "valid", TRAIN_DIR, df_valid),
    batch_size=16,
    shuffle=False,
)